<a href="https://colab.research.google.com/github/human-ai2025/Elo-Merchant-Recommendation/blob/master/EDA_feature_selection_elo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import mode

In [ ]:
#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Setting up worksapce directory 
%cd /content/drive/MyDrive/data 

/content/drive/MyDrive/data


In [ ]:
#refer:-https://www.youtube.com/watch?v=vOMtQ4ocMGI

@contextmanager
def timer(title):
    """ used to calculate time for each function"""
    t0 = time.time()
    yield
    print("{} - done in {:.000f}s".format(title, time.time() - t0))

In [ ]:
#https://www.kaggle.com/fabiendaniel/elo-world
#Function to load data into pandas and reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    #loop for alll the columns in the dataframe 
    for col in df.columns:
        #get the datatype of the column
        col_type = df[col].dtypes
        #if the data type is numeric then only start changing the datatype
        #as it isnt much helpful for other data types 
        if col_type in numerics:
            #stores the min value of the column 
            c_min = df[col].min()
            #stores the maximum value of the column
            c_max = df[col].max()
            #for int type numerics
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            #for float type numerics 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#Refer:-https://stackoverflow.com/questions/36631163/what-are-the-pros-and-cons-between-get-dummies-pandas-and-onehotencoder-sciki
def one_hot_encoder(df, nan_as_category=True):
    """used to create the one hot encoding of the categorical variables """
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    print("The columns on which one hot encoding is performed is ",categorical_columns)
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
'index', 'authorized_flag', 'card_id', 'city_id', 'category_1',
       'installments', 'category_3', 'merchant_category_id', 'merchant_id',
       'month_lag', 'purchase_amount', 'purchase_date', 'category_2',
       'state_id', 'subsector_id', 'merchant_group_id_merchants_t',
       'merchant_category_id_merchants_t', 'subsector_id_merchants_t',
       'numerical_1_merchants_t', 'numerical_2_merchants_t',
       'category_1_merchants_t', 'most_recent_sales_range_merchants_t',
       'most_recent_purchases_range_merchants_t', 'avg_sales_lag3_merchants_t',
       'avg_purchases_lag3_merchants_t', 'active_months_lag3_merchants_t',
       'avg_sales_lag6_merchants_t', 'avg_purchases_lag6_merchants_t',
       'active_months_lag6_merchants_t', 'avg_sales_lag12_merchants_t',
       'avg_purchases_lag12_merchants_t', 'active_months_lag12_merchants_t',
       'category_4_merchants_t', 'city_id_merchants_t', 'state_id_merchants_t',
       'category_2_merchants_t']

In [ ]:
def ground_on_card_id(df):
    """
    FUNCTION:
    To group the data on card id  

    ARGS:
    df is the data frame on which grouping needs to be performed 

    RETURNS:
    returns the data frame after grouping 
    
    """

    #Some feature engineering on date and time  
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >= 5).astype(int)

    #for id 
    col_id = ['city_id','merchant_id','state_id',
                            'subsector_id','merchant_group_id_merchants_t','state_id_merchants_t',
                            'merchant_category_id_merchants_t','subsector_id_merchants_t','city_id_merchants_t']
    #for numerical 
    col_numerical = ['month_lag', 'purchase_amount', 'month', 'hour', 'weekofyear', 'weekday', 'day',
                      'numerical_2_merchants_t','avg_sales_lag3_merchants_t',
                      'avg_purchases_lag3_merchants_t', 'active_months_lag3_merchants_t',
                      'avg_sales_lag6_merchants_t', 'avg_purchases_lag6_merchants_t',
                      'active_months_lag6_merchants_t', 'avg_sales_lag12_merchants_t',
                      'avg_purchases_lag12_merchants_t', 'active_months_lag12_merchants_t']

    aggs = {}
    for col in col_id:
        aggs[col] = ['nunique']

    for col in col_numerical:
        aggs[col] = ['nunique', 'mean', 'min', 'max']

    aggs['purchase_amount'] = ['sum', 'max', 'min', 'mean']
    aggs['installments'] = ['sum', 'max', 'mean']
    aggs['purchase_date'] = ['max', 'min']
    aggs['month_lag'] = ['max', 'min', 'mean']
    aggs['authorized_flag'] = ['mean']
    aggs['category_1'] = ['mean']
    aggs['category_2'] = ['mean']
    aggs['category_3'] = ['mean']
    aggs['category_1_merchants_t'] = ['mean']
    aggs['category_2_merchants_t'] = ['mean']
    aggs['category_4_merchants_t'] = ['mean']
    aggs['most_recent_sales_range_merchants_t'] = ['mean']
    aggs['most_recent_purchases_range_merchants_t'] = ['mean']

    df = df.reset_index().groupby('card_id').agg(aggs)

    # change column name
    df.columns = pd.Index([e[0] + "_" + e[1] for e in df.columns.tolist()])
    df.columns = ['new_' + c for c in df.columns]
    
    # reduce memory usage
    df = reduce_mem_usage(df)

    return df

In [ ]:
def transactions_data_imputations(df):
    """
    FUNCTION:
    Tohandle the missing values and inf values 

    ARGS:
    df is the dataframe to be imputed

    RETURNS:
    returns the data frame after imputations 
    
    """
    
    #for category 1 and authorized flag 
    #as they ahve no null values so directly encoding 
    df['category_1'] = df['category_1'].map({'Y': 0, 'N': 1}).astype(int)
    df['authorized_flag'] = df['authorized_flag'].map({'Y': 0, 'N': 1}).astype(int)

    #for category 3 
    #There are three categorical values with missing values 
    df['category_3'] = df['category_3'].fillna(3)
    df['category_3'] = df['category_3'].map({'A': 0, 'B': 1,'C': 2, 3:3}).astype(int)

    #for category_1_merchants_t_merchants_t
    # it contains 2 categorical and null values
    df['category_1_merchants_t'] = df['category_1_merchants_t'].fillna(2)
    df['category_1_merchants_t'] = df['category_1_merchants_t'].map({'Y': 0, 'N': 1,2:2}).astype(int)

    #for most_recent_sales_range_merchants_t_merchants_t and most_recent_purchases_range_merchants_t_merchants_t
    #it has A,B,C,D,E and null values 
    df['most_recent_sales_range_merchants_t'] = df['most_recent_sales_range_merchants_t'].fillna(5)
    df['most_recent_purchases_range_merchants_t'] = df['most_recent_purchases_range_merchants_t'].fillna(5)
    df['most_recent_sales_range_merchants_t'] = df['most_recent_sales_range_merchants_t'].map({'A': 0, 'B': 1,'C': 2, 'D': 3,'E' : 4, 5 : 5}).astype(int)
    df['most_recent_purchases_range_merchants_t'] = df['most_recent_purchases_range_merchants_t'].map({'A': 0, 'B': 1,'C': 2, 'D': 3,'E' : 4, 5 : 5}).astype(int)
    
    #for category_4_merchants_t_merchants_t
    #it has Y,N and null values 
    df['category_4_merchants_t'] = df['category_4_merchants_t'].fillna(2)
    df['category_4_merchants_t'] = df['category_4_merchants_t'].map({'Y': 0, 'N': 1, 2 : 2}).astype(int)

    #for category_2_merchants_t_merchants_t and category_2
    #it has 1.0,2.0,3.0,4.05.0 and null values 
    df['category_2_merchants_t'] = df['category_2_merchants_t'].fillna(6)
    df['category_2'] = df['category_2'].fillna(6)
    df['category_2_merchants_t'] = df['category_2_merchants_t'].map({1.0 : 1, 2.0 : 2, 3.0 : 3,4.0 : 4, 5.0 : 5, 6 : 6}).astype(int)
    df['category_2'] = df['category_2'].map({1.0 : 1, 2.0 : 2, 3.0 : 3,4.0 : 4, 5.0 : 5, 6 : 6}).astype(int)

    #for missing id we will use -1111 as filling value 
    #find the number of missing values 
    for col in df[['city_id','merchant_id','state_id',
                            'subsector_id','merchant_group_id_merchants_t','state_id_merchants_t',
                            'merchant_category_id_merchants_t','subsector_id_merchants_t','city_id_merchants_t']]:
                            df[col] = df[col].fillna(-1111)

    #imputing the inf values with max 
    features_inf = ["avg_purchases_lag3_merchants_t","avg_purchases_lag6_merchants_t","avg_purchases_lag12_merchants_t"]
    for col in features_inf:
        df.loc[df[col]==np.inf,col] = max(df.loc[df[col]!=np.inf,col])
        

    return df

In [ ]:
def merge_trans_merch(num_rows=None):
    """
    FUNCTION:
    To merge transactions and merchant return the dataframe

    ARGS:
    num_rows to indicate the number of rows to load 

    RETURNS:
    returns the merged data frame 
    
    """

    historical_transactions = pd.read_csv('historical_transactions.csv', nrows=num_rows)
    new_merchant_transactions = pd.read_csv('new_merchant_transactions.csv', nrows=num_rows)
    merchants = pd.read_csv('merchants.csv', nrows=num_rows)

    historical_transactions = reduce_mem_usage(historical_transactions)
    new_merchant_transactions = reduce_mem_usage(new_merchant_transactions)
    new_merchant_transactions = reduce_mem_usage(new_merchant_transactions)


    #The transactions and new merchat transactions has the same columns  
    transactions = pd.concat([historical_transactions, new_merchant_transactions], ignore_index=True)
    assert (transactions.shape[0] == historical_transactions.shape[0]+new_merchant_transactions.shape[0])

    merchants.columns = [col+"_merchants_t" if col!="merchant_id" else col for col in merchants.columns]
    df = pd.merge(transactions, merchants,how="left",on="merchant_id")
    del new_merchant_transactions
    del historical_transactions
    del transactions
    gc.collect()

    df = reduce_mem_usage(df)

    return df

In [ ]:
def merge_with_train_test(num_rows=None):
    """
    FUNCTION:
    To merge transactions and merchant return the dataframe

    ARGS:
    num_rows to indicate the number of rows to load 

    RETURNS:
    returns the merged data frame 
    
    """

    

    return df

In [ ]:
def generate_augmented_train_test(num_rows = None):
    """ Generate train and test augmented datasets i.e to join different dataframes """
    with timer("merge_trans_merch_data"):
      df = merge_trans_merch(num_rows).reset_index()
    with timer("imputation"):
      df = transactions_data_imputations(df)
    with timer("imputation"):
      df = ground_on_card_id(df)
    return df
    

In [ ]:
if __name__ == "__main__":
    df = generate_augmented_train_test(10)

Mem. usage decreased to  0.00 Mb (41.7% reduction)
Mem. usage decreased to  0.00 Mb (41.7% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
merge_trans_merch_data - done in 0s
imputation - done in 0s
Mem. usage decreased to  0.00 Mb (50.1% reduction)
imputation - done in 0s


In [ ]:
df

,new_city_id_nunique,new_merchant_id_nunique,new_state_id_nunique,new_subsector_id_nunique,new_merchant_group_id_merchants_t_nunique,new_state_id_merchants_t_nunique,new_merchant_category_id_merchants_t_nunique,new_subsector_id_merchants_t_nunique,new_city_id_merchants_t_nunique,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_month_nunique,new_month_mean,new_month_min,new_month_max,new_hour_nunique,new_hour_mean,new_hour_min,new_hour_max,new_weekofyear_nunique,new_weekofyear_mean,new_weekofyear_min,new_weekofyear_max,new_weekday_nunique,new_weekday_mean,new_weekday_min,new_weekday_max,new_day_nunique,new_day_mean,new_day_min,new_day_max,new_numerical_2_merchants_t_nunique,new_numerical_2_merchants_t_mean,new_numerical_2_merchants_t_min,new_numerical_2_merchants_t_max,...,new_active_months_lag3_merchants_t_min,new_active_months_lag3_merchants_t_max,new_avg_sales_lag6_merchants_t_nunique,new_avg_sales_lag6_merchants_t_mean,new_avg_sales_lag6_merchants_t_min,new_avg_sales_lag6_merchants_t_max,new_avg_purchases_lag6_merchants_t_nunique,new_avg_purchases_lag6_merchants_t_mean,new_avg_purchases_lag6_merchants_t_min,new_avg_purchases_lag6_merchants_t_max,new_active_months_lag6_merchants_t_nunique,new_active_months_lag6_merchants_t_mean,new_active_months_lag6_merchants_t_min,new_active_months_lag6_merchants_t_max,new_avg_sales_lag12_merchants_t_nunique,new_avg_sales_lag12_merchants_t_mean,new_avg_sales_lag12_merchants_t_min,new_avg_sales_lag12_merchants_t_max,new_avg_purchases_lag12_merchants_t_nunique,new_avg_purchases_lag12_merchants_t_mean,new_avg_purchases_lag12_merchants_t_min,new_avg_purchases_lag12_merchants_t_max,new_active_months_lag12_merchants_t_nunique,new_active_months_lag12_merchants_t_mean,new_active_months_lag12_merchants_t_min,new_active_months_lag12_merchants_t_max,new_installments_sum,new_installments_max,new_installments_mean,new_purchase_date_max,new_purchase_date_min,new_authorized_flag_mean,new_category_1_mean,new_category_2_mean,new_category_3_mean,new_category_1_merchants_t_mean,new_category_2_merchants_t_mean,new_category_4_merchants_t_mean,new_most_recent_sales_range_merchants_t_mean,new_most_recent_purchases_range_merchants_t_mean
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_415bb3a509,4,4,2,3,1,1,1,1,1,2,1,1.25,-2.349609,-0.551270,-0.671875,-0.587402,2,3.250000,3,4,3,13.75,9,18,3,12.25000,10,17,4,2.750000,0,6,4,15.750000,7,26,0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,4,1,1,2018-04-26 14:08:44,2018-03-07 09:43:21,0,0.750000,2.250000,1,2,6,2,5,5
C_ID_4e6213e9bc,3,8,2,3,1,1,1,1,1,0,-11,-7.00,-7.175781,-0.657227,-0.737793,-0.717773,7,5.800781,2,11,8,12.50,0,22,10,22.90625,8,46,6,3.900391,1,6,10,14.101562,1,25,0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,0,2018-02-24 08:45:05,2017-03-10 01:14:19,0,1.000000,1.000000,0,2,6,2,5,5
C_ID_ef55cf8d4b,3,6,2,6,1,1,1,1,1,2,1,1.50,2.214844,5.265625,-0.659668,0.369141,2,3.500000,3,4,5,16.00,8,21,4,13.00000,11,14,4,3.000000,0,5,6,13.500000,2,28,0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,6,1,1,2018-04-07 18:37:40,2018-03-17 18:10:41,0,0.833496,1.833008,1,2,6,2,5,5


In [ ]:
df.isnull().sum(axis = 0)

new_city_id_nunique                                 0
new_merchant_id_nunique                             0
new_state_id_nunique                                0
new_subsector_id_nunique                            0
new_merchant_group_id_merchants_t_nunique           0
                                                   ..
new_category_1_merchants_t_mean                     0
new_category_2_merchants_t_mean                     0
new_category_4_merchants_t_mean                     0
new_most_recent_sales_range_merchants_t_mean        0
new_most_recent_purchases_range_merchants_t_mean    0
Length: 90, dtype: int64

In [ ]:
#Gives the name of all the variables with missing data 

columns_with_na = [var for var in df.columns if df[var].isnull().mean()  > 0]

In [ ]:
len(columns_with_na)

30

In [ ]:
#lets find out the percentage of observations missing per variable

#calculate the percentage of missing 
data_na = df[columns_with_na].isnull().mean()

#transform the array to dataframe 
data_na = pd.DataFrame(data_na.reset_index())

#add names to the dataframe 
data_na.columns = ['col','percentage_na']

#oreder the dataframe acc to percentage 
data_na.sort_values(by = 'percentage_na',ascending = False, inplace = True)

#show
data_na

,col,percentage_na
0,new_numerical_2_merchants_t_mean,1.0
1,new_numerical_2_merchants_t_min,1.0
28,new_active_months_lag12_merchants_t_min,1.0
27,new_active_months_lag12_merchants_t_mean,1.0
26,new_avg_purchases_lag12_merchants_t_max,1.0
25,new_avg_purchases_lag12_merchants_t_min,1.0
24,new_avg_purchases_lag12_merchants_t_mean,1.0
23,new_avg_sales_lag12_merchants_t_max,1.0
22,new_avg_sales_lag12_merchants_t_min,1.0
21,new_avg_sales_lag12_merchants_t_mean,1.0


In [ ]:
with timer("transactions_data_imputations_data"):
      df = transactions_data_imputations()
        
    with timer("split train & test"):
        train_df = df[df['target'].notnull()]
        test_df = df[df['target'].isnull()]
        del test_df['target']
        del df
        gc.collect()
    with timer("Save train and test files"):
        train_df.to_csv('/content/drive/MyDrive/data/augmented_train.csv', index=False)
        test_df.to_csv('/content/drive/MyDrive/data/augmented_test.csv', index=False)

In [ ]:
def lgb_train_fn(train_df, target, trn_cols,  n_fold):
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=4590)
    # initialise out of fold preds to 0s.
    oof = np.zeros(len(train_df))

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df,train_df['outliers'].values)):
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][trn_cols], label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train_df.iloc[val_idx][trn_cols], label=target.iloc[val_idx])

        num_round = 10000
        clf = lgb.train(lgb_param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 200)
        oof[val_idx] = clf.predict(train_df.iloc[val_idx][trn_cols], num_iteration=clf.best_iteration)

    print(np.sqrt(mean_squared_error(oof, target)), 'CV score')
    return np.sqrt(mean_squared_error(oof, target))

In [ ]:
for c in cols_to_add:
    lgb_cols = final_cols + [c]
    print(len(lgb_cols), 'lg_cols', c)
    score = lgb_train_fn(x, y, lgb_cols, 5)
    delta = base_score - score
    fe_d[c] = delta
    if delta > 0:
        base_score = score
        selected_cols.append(c)
        print('Selected cols', c)
        print('Selected col delta', delta)
        print(' score with col', score)
        np.save('selecte_cols_extra', selected_cols)
        final_cols = final_cols + [c]